<a href="https://colab.research.google.com/github/julika333/TMS-DS/blob/main/TMS_nb/Lesson_27.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Установка библиотек:
Убедитесь, что у вас установлены библиотеки hyperopt и tensorflow. Если нет, установите их с помощью команды:

In [ ]:
!pip install hyperopt tensorflow


  Using cached hyperopt-0.2.7-py2.py3-none-any.whl (1.6 MB)
  Using cached future-0.18.3.tar.gz (840 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached cloudpickle-2.2.1-py3-none-any.whl (25 kB)
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
  Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
  Created wheel for future: filename=future-0.18.3-py3-none-any.whl size=492053 sha256=9176799bf0b730d852c86a8758feae140d5fac5e36b73b455943a606c0d20ec2
  Stored in directory: c:\users\sidromnik\appdata\local\pip\cache\wheels\da\19\ca\9d8c44cd311a955509d7e13da3f0bea42400c469ef825b580b
Successfully built future
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pydantic 2.1.1 requires typing-extensions>=4.6.1, but you have typing-extensions 4.5.0 which is incompatible.
pydantic-core 2.4.0 requires typing-extensions!=4.7.0,>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.


Импорт библиотек:

In [ ]:
import hyperopt
from hyperopt import fmin, tpe, hp
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential


Загрузка данных:

In [ ]:
# Загрузка и предобработка данных
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0


Определение функции для оптимизации:

In [ ]:
def objective(params):
    model = Sequential([
        Flatten(input_shape=(32, 32, 3)),
        Dense(params['units'], activation='relu'),
        Dense(10, activation='softmax')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=params['learning_rate']),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test), verbose=0)
    val_accuracy = history.history['val_accuracy'][-1]

    return -val_accuracy  # Минимизация, так как Hyperopt ищет минимум функции


Определение пространства поиска:

In [ ]:
space = {
    'units': hp.choice('units', [32, 64, 128]),  # Количество нейронов в слое
    'learning_rate': hp.loguniform('learning_rate', -5, -1)  # Логарифмический выбор скорости обучения
}


Запуск поиска:

In [ ]:
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10)  # Количество экспериментов


100%|██████████| 10/10 [07:06<00:00, 42.62s/trial, best loss: -0.17319999635219574]


В этом примере мы использовали библиотеку Hyperopt для оптимизации гиперпараметров нейронной сети для задачи классификации CIFAR-10 с использованием TensorFlow. Мы определили функцию objective, которую Hyperopt будет минимизировать, и пространство поиска гиперпараметров. Затем мы запустили поиск с помощью алгоритма tpe.suggest и ограничили количество экспериментов до 10.

примеры использования библиотеки Scikit-learn для поиска гиперпараметров нейронной сети на задаче классификации изображений CIFAR-10 с использованием TensorFlow:


## Grid Search:

In [ ]:
# !pip install scikeras

In [ ]:
import numpy as np
from sklearn.datasets import make_classification
from tensorflow import keras

from scikeras.wrappers import KerasClassifier


X, y = make_classification(1000, 20, n_informative=10, random_state=0)
X = X.astype(np.float32)
y = y.astype(np.int64)

def get_model(hidden_layer_dim, meta):
    # note that meta is a special argument that will be
    # handed a dict containing input metadata
    n_features_in_ = meta["n_features_in_"]
    X_shape_ = meta["X_shape_"]
    n_classes_ = meta["n_classes_"]

    model = keras.models.Sequential()
    model.add(keras.layers.Dense(n_features_in_, input_shape=X_shape_[1:]))
    model.add(keras.layers.Activation("relu"))
    model.add(keras.layers.Dense(hidden_layer_dim))
    model.add(keras.layers.Activation("relu"))
    model.add(keras.layers.Dense(n_classes_))
    model.add(keras.layers.Activation("softmax"))
    return model

clf = KerasClassifier(
    get_model,
    loss="sparse_categorical_crossentropy",
    hidden_layer_dim=100,
)

clf.fit(X, y)
y_proba = clf.predict_proba(X)








32/32 [==============================] - 0s 839us/step


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


pipe = Pipeline([
    ('scale', StandardScaler()),
    ('clf', clf),
])

pipe.fit(X, y)
y_proba = pipe.predict_proba(X)

32/32 [==============================] - 0s 742us/step


In [ ]:
from sklearn.model_selection import GridSearchCV


params = {
    "hidden_layer_dim": [50, 100, 200],
    "loss": ["sparse_categorical_crossentropy"],
    "optimizer": ["adam", "sgd"],
    "optimizer__learning_rate": [0.0001, 0.001, 0.1],
}
gs = GridSearchCV(clf, params, refit=False, cv=3, scoring='accuracy')

gs.fit(X, y)
print(gs.best_score_, gs.best_params_)

11/11 [==============================] - 0s 900us/step
0.7360114605623588 {'hidden_layer_dim': 100, 'loss': 'sparse_categorical_crossentropy', 'optimizer': 'sgd', 'optimizer__learning_rate': 0.1}


## Randomized Search:

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

params = {
    "hidden_layer_dim": [50, 100, 200],
    "loss": ["sparse_categorical_crossentropy"],
    "optimizer": ["adam", "sgd"],
    "optimizer__learning_rate": [0.0001, 0.001, 0.1],
}
gs = RandomizedSearchCV(clf, params, refit=False, cv=3, scoring='accuracy')

gs.fit(X, y)
print(gs.best_score_, gs.best_params_)

11/11 [==============================] - 0s 700us/step
0.7410134685583788 {'optimizer__learning_rate': 0.1, 'optimizer': 'adam', 'loss': 'sparse_categorical_crossentropy', 'hidden_layer_dim': 100}


В этих примерах мы использовали библиотеку Scikit-learn для поиска гиперпараметров нейронной сети для задачи классификации CIFAR-10 с использованием TensorFlow. Мы определили функцию create_model, которая создает модель нейронной сети, и параметры поиска param_dist (для Randomized Search) и param_grid (для Grid Search). Затем мы создали объекты RandomizedSearchCV и GridSearchCV и выполнили поиск с использованием кросс-валидации.

пример использования библиотеки Optuna для настройки гиперпараметров нейронной сети на задаче классификации изображений CIFAR-10 с использованием TensorFlow:

In [ ]:
# !pip install optuna

In [ ]:
import optuna
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam

# Load and preprocess CIFAR-10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Define the objective function
def objective(trial):
    units = trial.suggest_categorical('units', [32, 64, 128])
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)

    model = Sequential([
        Flatten(input_shape=(32, 32, 3)),
        Dense(units, activation='relu'),
        Dense(10, activation='softmax')
    ])

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test), verbose=0)
    val_accuracy = history.history['val_accuracy'][-1]

    return val_accuracy

# Create a study object and optimize
study = optuna.create_study(direction='maximize')  # 'maximize' for accuracy, 'minimize' for loss
study.optimize(objective, n_trials=10)

# Print the best parameters and value
print("Best parameters found: ", study.best_params)
print("Best value found: {:.2f}%".format(study.best_value * 100))


[I 2023-08-30 14:23:47,264] A new study created in memory with name: no-name-fec5a293-d11f-44fa-8076-fcb8ce6672fd
C:\Users\Sidromnik\AppData\Local\Temp\ipykernel_11752\3713110107.py:15: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
[I 2023-08-30 14:24:14,730] Trial 0 finished with value: 0.3935000002384186 and parameters: {'units': 32, 'learning_rate': 1.7722962087941887e-05}. Best is trial 0 with value: 0.3935000002384186.
[I 2023-08-30 14:25:27,520] Trial 1 finished with value: 0.41110000014305115 and parameters: {'units': 128, 'learning_rate': 1.07200310649519e-05}. Best is trial 1 with value: 0.41110000014305115.
[I 2023-08-30 14:25:54,069] Trial 2 finished with value: 0.37299999594688416 and parameters: {'units': 32, 'learning_rate': 1.0546417474900653e-

Best parameters found:  {'units': 128, 'learning_rate': 8.665223022938086e-05}
Best value found: 46.53%


In [ ]:
# !pip install keras-tuner

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from kerastuner.tuners import RandomSearch

# Загрузка данных CIFAR-10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Определение модели для настройки гиперпараметров
def build_model(hp):
    model = tf.keras.Sequential()
    model.add(Conv2D(filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
                     kernel_size=hp.Choice('conv_1_kernel', values=[3, 5]),
                     activation='relu', input_shape=(32, 32, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    for i in range(hp.Int('num_conv_layers', 1, 3)):
        model.add(Conv2D(filters=hp.Int(f'conv_{i+2}_filter', min_value=32, max_value=128, step=16),
                         kernel_size=hp.Choice(f'conv_{i+2}_kernel', values=[3, 5]),
                         activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(units=hp.Int('dense_units', min_value=32, max_value=512, step=32),
                    activation='relu'))
    model.add(Dense(10, activation='softmax'))

    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Создание объекта Keras Tuner RandomSearch
tuner = RandomSearch(build_model,
                     objective='val_accuracy',
                     max_trials=5,  # Количество случайных комбинаций гиперпараметров для попыток
                     directory='my_dir',  # Папка для сохранения результатов настройки
                     project_name='cifar10')

# Поиск оптимальных гиперпараметров
tuner.search(x_train, y_train, epochs=10, validation_split=0.1)

# Вывод результатов настройки
tuner.results_summary()

# Получение лучшей модели
best_model = tuner.get_best_models(num_models=1)[0]


Trial 5 Complete [00h 05m 56s]
val_accuracy: 0.6844000220298767

Best val_accuracy So Far: 0.7070000171661377
Total elapsed time: 00h 20m 44s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in my_dir\cifar10
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0 summary
Hyperparameters:
conv_1_filter: 48
conv_1_kernel: 3
num_conv_layers: 1
conv_2_filter: 32
conv_2_kernel: 3
dense_units: 288
learning_rate: 0.001
Score: 0.7070000171661377

Trial 4 summary
Hyperparameters:
conv_1_filter: 48
conv_1_kernel: 5
num_conv_layers: 1
conv_2_filter: 80
conv_2_kernel: 5
dense_units: 256
learning_rate: 0.001
conv_3_filter: 64
conv_3_kernel: 3
conv_4_filter: 112
conv_4_kernel: 3
Score: 0.6844000220298767

Trial 2 summary
Hyperparameters:
conv_1_filter: 112
conv_1_kernel: 5
num_conv_layers: 1
conv_2_filter: 96
conv_2_kernel: 5
dense_units: 224
learning_rate: 0.01
conv_3_filter: 128
conv_3_kernel: 3
conv_4_filter: 96
conv_4_kernel: 5
Score: 0.38199999928474426

Tr

In [ ]:
# !pip install bayesian-optimization

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from bayes_opt import BayesianOptimization

# Загрузка данных CIFAR-10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Определение функции для оптимизации
def keras_cnn(learning_rate, conv_1_filter, conv_1_kernel,
              num_conv_layers, dense_units):
    model = tf.keras.Sequential()
    model.add(Conv2D(filters=int(conv_1_filter), kernel_size=int(conv_1_kernel),
                     activation='relu', input_shape=(32, 32, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    for i in range(int(num_conv_layers)):
        model.add(Conv2D(filters=32, kernel_size=3, activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(units=int(dense_units), activation='relu'))
    model.add(Dense(10, activation='softmax'))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(x_train, y_train, epochs=5, validation_split=0.1, verbose=0)
    val_accuracy = history.history['val_accuracy'][-1]
    return val_accuracy

# Определение диапазонов гиперпараметров для оптимизации
pbounds = {'learning_rate': (1e-4, 1e-2),
           'conv_1_filter': (32, 128),
           'conv_1_kernel': (3, 5),
           'num_conv_layers': (1, 3),
           'dense_units': (32, 512)}

# Создание объекта BayesianOptimization
optimizer = BayesianOptimization(f=keras_cnn, pbounds=pbounds, verbose=2, random_state=1)

# Запуск оптимизации
optimizer.maximize(init_points=5, n_iter=10)

# Вывод результатов оптимизации
print("Best parameters:", optimizer.max['params'])
print("Best validation accuracy:", optimizer.max['target'])


|   iter    |  target   | conv_1... | conv_1... | dense_... | learni... | num_co... |
-------------------------------------------------------------------------------------
| 1         | 0.5894    | 72.03     | 4.441     | 32.05     | 0.003093  | 1.294     |
| 2         | 0.5804    | 40.86     | 3.373     | 197.9     | 0.004028  | 2.078     |
| 3         | 0.0986    | 72.24     | 4.37      | 130.1     | 0.008793  | 1.055     |
| 4         | 0.6882    | 96.36     | 3.835     | 300.2     | 0.00149   | 1.396     |
| 5         | 0.0976    | 108.9     | 4.937     | 182.4     | 0.006954  | 2.753     |
| 6         | 0.5696    | 96.22     | 4.464     | 300.5     | 0.0027    | 2.041     |
| 7         | 0.4946    | 97.73     | 3.296     | 298.7     | 0.007028  | 1.133     |
| 8         | 0.097     | 94.69     | 3.613     | 299.9     | 0.009769  | 1.268     |
| 9         | 0.097     | 96.58     | 4.594     | 299.3     | 0.004438  | 1.269     |
| 10        | 0.0976    | 96.88     | 3.587     | 300.

In [ ]:
# conda create -n SMAC python=3.10
# conda activate SMAC
# conda install gxx_linux-64 gcc_linux-64 swig

In [ ]:
# import numpy as np
# import tensorflow as tf
# from tensorflow.keras.datasets import cifar10
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
# from smac.facade.func_facade import fmin_smac

# # Загрузка данных CIFAR-10
# (x_train, y_train), (x_test, y_test) = cifar10.load_data()
# x_train, x_test = x_train / 255.0, x_test / 255.0

# # Определение функции для оптимизации
# def keras_cnn(learning_rate, conv_1_filter, conv_1_kernel,
#               num_conv_layers, dense_units):
#     model = tf.keras.Sequential()
#     model.add(Conv2D(filters=int(conv_1_filter), kernel_size=int(conv_1_kernel),
#                      activation='relu', input_shape=(32, 32, 3)))
#     model.add(MaxPooling2D(pool_size=(2, 2)))
#     for i in range(int(num_conv_layers)):
#         model.add(Conv2D(filters=32, kernel_size=3, activation='relu'))
#         model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Flatten())
#     model.add(Dense(units=int(dense_units), activation='relu'))
#     model.add(Dense(10, activation='softmax'))

#     model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
#                   loss='sparse_categorical_crossentropy',
#                   metrics=['accuracy'])

#     history = model.fit(x_train, y_train, epochs=5, validation_split=0.1, verbose=0)
#     val_accuracy = history.history['val_accuracy'][-1]
#     return -val_accuracy

# # Определение диапазонов гиперпараметров для оптимизации
# param_space = {'learning_rate': ('log-uniform', 1e-4, 1e-2),
#                'conv_1_filter': (32, 128),
#                'conv_1_kernel': (3, 5),
#                'num_conv_layers': (1, 3),
#                'dense_units': (32, 512)}

# # Запуск оптимизации с использованием SMAC
# best_config, _ = fmin_smac(func=keras_cnn,  # Функция для оптимизации
#                            x0=[0.001, 64, 3, 2, 128],  # Начальные значения гиперпараметров
#                            bounds=[(1e-4, 1e-2), (32, 128), (3, 5), (1, 3), (32, 512)],  # Диапазоны гиперпараметров
#                            maxfun=20,  # Количество итераций
#                            rng=np.random.RandomState(42))

# # Вывод наилучших параметров
# print("Best parameters:", best_config)
